In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import os
import shutil

def start():
    project_name = input("Enter the new Django project name: ").strip()
    
    if not project_name:
        print("Project name cannot be empty.")
        return

    if os.path.exists(project_name):
        print(
            f"Directory '{project_name}' already exists. "
            "Please choose a different name or remove the existing directory."
        )
        return

    shutil.copytree('django_template', project_name)
    print(f"Django template copied to '{project_name}' directory.")

    return project_name


In [28]:
project_name = start()


Enter the new Django project name:  my_new_app


Directory 'my_new_app' already exists. Please choose a different name or remove the existing directory.


In [30]:
print(project_name)

None


In [31]:
project_name = "my_new_app" 

### AI Test

In [32]:
from pathlib import Path
import tools

project_path = Path(project_name)
agent_tools = tools.AgentTools(project_path)

agent_tools.list_files()[:5]


['.python-version',
 '.venv/.gitignore',
 '.venv/.lock',
 '.venv/CACHEDIR.TAG',
 '.venv/bin/activate']

In [33]:
import tools
from pathlib import Path

project_path = Path(project_name)  # project_name was created by start()
agent_tools = tools.AgentTools(project_path)

agent_tools


AgentTools(project_path=PosixPath('my_new_app'))

In [34]:
print(agent_tools.read_file("myproject/settings.py")[:200])

"""
Django settings for myproject project.

Generated by 'django-admin startproject' using Django 6.0.

For more information on this file, see
https://docs.djangoproject.com/en/6.0/topics/settings/

F


In [35]:
agent_tools.write_file(
    "TEST_AGENT.txt",
    "Hello from the agent tools!"
)

'Wrote 27 characters to TEST_AGENT.txt'

In [36]:
print(agent_tools.read_file("TEST_AGENT.txt"))

Hello from the agent tools!


In [37]:
files = agent_tools.list_files()
files[:10]

['.python-version',
 '.venv/.gitignore',
 '.venv/.lock',
 '.venv/CACHEDIR.TAG',
 '.venv/bin/activate',
 '.venv/bin/activate.bat',
 '.venv/bin/activate.csh',
 '.venv/bin/activate.fish',
 '.venv/bin/activate.nu',
 '.venv/bin/activate.ps1']

In [38]:
agent_tools.grep("INSTALLED_APPS")

['myproject/settings.py:33: INSTALLED_APPS = [',
 'myproject/__pycache__/settings.cpython-312.pyc:30: ALLOWED_HOSTS\x0eINSTALLED_APPS',
 '.venv/lib/python3.12/site-packages/django/__init__.py:24: apps.populate(settings.INSTALLED_APPS)',
 '.venv/lib/python3.12/site-packages/django/test/utils.py:486: # in case it raises an exception because INSTALLED_APPS is invalid.',
 '.venv/lib/python3.12/site-packages/django/test/utils.py:487: if "INSTALLED_APPS" in self.options:',
 '.venv/lib/python3.12/site-packages/django/test/utils.py:489: apps.set_installed_apps(self.options["INSTALLED_APPS"])',
 '.venv/lib/python3.12/site-packages/django/test/utils.py:511: if "INSTALLED_APPS" in self.options:',
 '.venv/lib/python3.12/site-packages/django/test/utils.py:608: Useful when you override `INSTALLED_APPS`, e.g. if you exclude `auth` app,',
 '.venv/lib/python3.12/site-packages/django/test/testcases.py:1157: setting="INSTALLED_APPS",',
 '.venv/lib/python3.12/site-packages/django/test/testcases.py:1170: s

In [39]:
agent_tools.run_bash("ls")

'README.md\nTEST_AGENT.txt\nmanage.py\nmyapp\nmyproject\npyproject.toml\ntemplates\nuv.lock\n'

In [40]:
agent_tools.run_bash("python manage.py runserver")

'Blocked: runserver commands are not allowed from the agent.'

### Connect Agent to the LLM

In [41]:
from openai import OpenAI

from toyaikit.tools import Tools
from toyaikit.chat import IPythonChatInterface
from toyaikit.llm import OpenAIClient
from toyaikit.chat.runners import OpenAIResponsesRunner


#### Create the tools registry

In [42]:
tools_obj = Tools()
tools_obj.add_tools(agent_tools)

tools_obj


In [43]:
DEVELOPER_PROMPT = """
You are a coding agent.
For now, you can only READ files and explain them.
Do not modify any files yet.
"""


#### Create the runner (this connects everything)

In [44]:
chat_interface = IPythonChatInterface()
llm_client = OpenAIClient(client=OpenAI())

runner = OpenAIResponsesRunner(
    tools=tools_obj,
    developer_prompt=DEVELOPER_PROMPT,
    chat_interface=chat_interface,
    llm_client=llm_client,
)


##### Test the agent

In [48]:
from openai import OpenAI
from toyaikit.tools import Tools
from toyaikit.llm import OpenAIClient
from toyaikit.chat.runners import OpenAIResponsesRunner


In [49]:
tools_obj = Tools()
tools_obj.add_tools(agent_tools)

DEVELOPER_PROMPT = """
You are a coding agent.
For now, you can only READ files and explain them.
Do not modify any files yet.
"""

runner = OpenAIResponsesRunner(
    tools=tools_obj,
    developer_prompt=DEVELOPER_PROMPT,
    llm_client=OpenAIClient(client=OpenAI()),
    chat_interface=None,   # 🔴 IMPORTANT: disable interactive mode
)


In [51]:
class OneShotChatInterface:
    def input(self):
        return "stop"  # immediately end the loop

    def display(self, message):
        print(message)


In [52]:
from openai import OpenAI
from toyaikit.tools import Tools
from toyaikit.llm import OpenAIClient
from toyaikit.chat.runners import OpenAIResponsesRunner

tools_obj = Tools()
tools_obj.add_tools(agent_tools)

DEVELOPER_PROMPT = """
You are a coding agent.
For now, you can only READ files and explain them.
Do not modify any files yet.
"""

runner = OpenAIResponsesRunner(
    tools=tools_obj,
    developer_prompt=DEVELOPER_PROMPT,
    chat_interface=OneShotChatInterface(),  # ✅ SAFE FIX
    llm_client=OpenAIClient(client=OpenAI()),
)


In [53]:
runner.run(
    "Read myproject/settings.py and summarize what this Django project is using."
)


Chat ended.


LoopResult(new_messages=['R', 'e', 'a', 'd', ' ', 'm', 'y', 'p', 'r', 'o', 'j', 'e', 'c', 't', '/', 's', 'e', 't', 't', 'i', 'n', 'g', 's', '.', 'p', 'y', ' ', 'a', 'n', 'd', ' ', 's', 'u', 'm', 'm', 'a', 'r', 'i', 'z', 'e', ' ', 'w', 'h', 'a', 't', ' ', 't', 'h', 'i', 's', ' ', 'D', 'j', 'a', 'n', 'g', 'o', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'i', 's', ' ', 'u', 's', 'i', 'n', 'g', '.'], all_messages=['R', 'e', 'a', 'd', ' ', 'm', 'y', 'p', 'r', 'o', 'j', 'e', 'c', 't', '/', 's', 'e', 't', 't', 'i', 'n', 'g', 's', '.', 'p', 'y', ' ', 'a', 'n', 'd', ' ', 's', 'u', 'm', 'm', 'a', 'r', 'i', 'z', 'e', ' ', 'w', 'h', 'a', 't', ' ', 't', 'h', 'i', 's', ' ', 'D', 'j', 'a', 'n', 'g', 'o', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'i', 's', ' ', 'u', 's', 'i', 'n', 'g', '.'], tokens=TokenUsage(model='gpt-4o-mini', input_tokens=0, output_tokens=0), cost=CostInfo(input_cost=Decimal('0.000'), output_cost=Decimal('0.0'), total_cost=Decimal('0.000')), last_message='')

In [54]:
DEVELOPER_PROMPT = """
You are a coding agent.

You are allowed to:
- Read files
- Modify ONLY the file README.md

You are NOT allowed to modify any other files.

Task behavior:
- If asked to change something, first read README.md
- Then write an improved version back to README.md
- Keep changes simple and safe

If the request is outside README.md, refuse politely.
"""


In [55]:
runner = OpenAIResponsesRunner(
    tools=tools_obj,
    developer_prompt=DEVELOPER_PROMPT,
    chat_interface=OneShotChatInterface(),  # safe, non-interactive
    llm_client=OpenAIClient(client=OpenAI()),
)


In [57]:
agent_tools.list_files()


['.python-version',
 '.venv/.gitignore',
 '.venv/.lock',
 '.venv/CACHEDIR.TAG',
 '.venv/bin/activate',
 '.venv/bin/activate.bat',
 '.venv/bin/activate.csh',
 '.venv/bin/activate.fish',
 '.venv/bin/activate.nu',
 '.venv/bin/activate.ps1',
 '.venv/bin/activate_this.py',
 '.venv/bin/deactivate.bat',
 '.venv/bin/django-admin',
 '.venv/bin/pydoc.bat',
 '.venv/bin/python',
 '.venv/bin/python3',
 '.venv/bin/python3.12',
 '.venv/bin/sqlformat',
 '.venv/lib/python3.12/site-packages/__pycache__/_virtualenv.cpython-312.pyc',
 '.venv/lib/python3.12/site-packages/_virtualenv.pth',
 '.venv/lib/python3.12/site-packages/_virtualenv.py',
 '.venv/lib/python3.12/site-packages/asgiref-3.11.0.dist-info/INSTALLER',
 '.venv/lib/python3.12/site-packages/asgiref-3.11.0.dist-info/METADATA',
 '.venv/lib/python3.12/site-packages/asgiref-3.11.0.dist-info/RECORD',
 '.venv/lib/python3.12/site-packages/asgiref-3.11.0.dist-info/REQUESTED',
 '.venv/lib/python3.12/site-packages/asgiref-3.11.0.dist-info/WHEEL',
 '.venv/l

In [63]:
agent_tools.list_files("django_template")


[]

In [64]:
agent_tools.list_files("myproject")

['myproject/__init__.py',
 'myproject/__pycache__/__init__.cpython-312.pyc',
 'myproject/__pycache__/settings.cpython-312.pyc',
 'myproject/__pycache__/urls.cpython-312.pyc',
 'myproject/asgi.py',
 'myproject/settings.py',
 'myproject/urls.py',
 'myproject/wsgi.py']

In [65]:
settings_text = agent_tools.read_file("myproject/settings.py")
print(settings_text[:1500])

"""
Django settings for myproject project.

Generated by 'django-admin startproject' using Django 6.0.

For more information on this file, see
https://docs.djangoproject.com/en/6.0/topics/settings/

For the full list of settings and their values, see
https://docs.djangoproject.com/en/6.0/ref/settings/
"""

from pathlib import Path

# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(__file__).resolve().parent.parent


# Quick-start development settings - unsuitable for production
# See https://docs.djangoproject.com/en/6.0/howto/deployment/checklist/

# SECURITY WARNING: keep the secret key used in production secret!
SECRET_KEY = "django-insecure-wuk=tb3=o-m&gs+@u!rsxugd@nk-w#kfk9^a!^p0+zyo4m_kxc"

# SECURITY WARNING: don't run with debug turned on in production!
DEBUG = True

ALLOWED_HOSTS = []


# Application definition

INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.session

### Build the prompt explicitly for LLM

In [66]:
prompt = f"""
You are analyzing a Django project's settings.py file.

Summarize the following in bullet points:
- Django version and environment
- Installed apps
- Database configuration
- Middleware highlights
- Security-related settings
- Deployment readiness (DEBUG, ALLOWED_HOSTS, static/media)

Here is the file content:

{settings_text}
"""


#### Call the runner

In [67]:
response = runner.loop(prompt=prompt)
print(response.new_messages[-1].content)


[ResponseOutputText(annotations=[], text="Here's a summary of the provided Django project's settings:\n\n- **Django Version and Environment**\n  - Django version: 6.0\n  - Environment: Development (DEBUG is set to True)\n\n- **Installed Apps**\n  - Built-in apps:\n    - `django.contrib.admin`\n    - `django.contrib.auth`\n    - `django.contrib.contenttypes`\n    - `django.contrib.sessions`\n    - `django.contrib.messages`\n    - `django.contrib.staticfiles`\n  - Custom app:\n    - `myapp`\n\n- **Database Configuration**\n  - Database engine: SQLite (`django.db.backends.sqlite3`)\n  - Database name: `db.sqlite3` located in the base directory\n\n- **Middleware Highlights**\n  - Security Middleware: `django.middleware.security.SecurityMiddleware`\n  - Session Middleware: `django.contrib.sessions.middleware.SessionMiddleware`\n  - CSRF Protection: `django.middleware.csrf.CsrfViewMiddleware`\n  - Authentication Middleware: `django.contrib.auth.middleware.AuthenticationMiddleware`\n  - Click